Use **md_solutions.py**; download all solution info, reformat and merge with compound info from Guillermo

In [26]:
import md_solutions as mds

id_list = list(range(1,10)) #1,7000 to encompass all solutions as far as I can tell

composition_df = mds.get_solutions(id_list) 

 56%|█████▌    | 5/9 [00:00<00:00, 13.33it/s]

Request failed with status code: 404


100%|██████████| 9/9 [00:00<00:00, 14.62it/s]


In [ ]:
# Some formatting
df = composition_df.copy()
df['solution_id'] = df['solution_id'].str[0]
df['solution_name'] = df['solution_name'].str[0]
df['steps'] = df['steps'].str[0].str[0]

#df.to_csv('~/Desktop/MediaDive_solutions.csv', index=False)
df

,solution_id,solution_name,component,component_id,component_gl,sub_sol,sub_component,sub_component_id,steps
0,1,Main sol. 1,"[Peptone, Meat extract, Agar, Distilled water]","[1, 2, 3, 4]","[5, 3, 15, None]",[],[],[],"{'recipe_order': None, 'step': 'Adjust pH to 7..."
1,3,Main sol. 1a,"[Peptone, Meat extract, Agar, Distilled water]","[1, 2, 3, 4]","[5, 3, 15, None]",[],[],[],"{'recipe_order': None, 'step': 'Adjust pH to 7..."
2,4,Main sol. 2,[Urea],[339],[20],[1],"[Peptone, Meat extract, Agar, Distilled water]","[1, 2, 3, 4]","{'recipe_order': None, 'step': 'To medium 1 ad..."


Merge w/ compound info

In [27]:
import pandas as pd

compounds = pd.read_csv("~/Desktop/MediaDive_ingredients.csv") # From Guillermo
solutions = pd.read_csv("~/Desktop/MediaDive_solutions.csv") # From above

solutions.head()

,solution_id,solution_name,component,component_id,component_gl,sub_sol,sub_component,sub_component_id,steps
0,1,Main sol. 1,"['Peptone', 'Meat extract', 'Agar', 'Distilled...","[1, 2, 3, 4]","[5, 3, 15, None]",[],[],[],"{'recipe_order': None, 'step': 'Adjust pH to 7..."
1,3,Main sol. 1a,"['Peptone', 'Meat extract', 'Agar', 'Distilled...","[1, 2, 3, 4]","[5, 3, 15, None]",[],[],[],"{'recipe_order': None, 'step': 'Adjust pH to 7..."
2,4,Main sol. 2,['Urea'],[339],[20],[1],"['Peptone', 'Meat extract', 'Agar', 'Distilled...","[1, 2, 3, 4]","{'recipe_order': None, 'step': 'To medium 1 ad..."
3,5,Main sol. 3,"['Glucose', 'Mannitol', 'CaCl2 x 2 H2O', 'MgSO...","[5, 6, 7, 8, 9, 10, 11, 12, 13, 3, 4]","[5, 5, 0.1, 0.1, 0.005, 0.9, 0.1, 0.01, 5, 15,...",[],[],[],"{'recipe_order': None, 'step': 'Adjust pH to 7..."
4,6,Main sol. 6,"['K2HPO4', 'KH2PO4', 'MgSO4 x 7 H2O', 'CaCl2 x...","[10, 11, 8, 7, 12, 14, 15, 3, 4]","[0.8, 0.2, 0.5, 0.05, 0.01, 0.001, 20, 15, None]",[],[],[],NaN


In [28]:
# Copy solution dataframe
sol_df = solutions.copy()
sol_df = sol_df[["solution_id","component_id","sub_component_id"]]

# Remove brackets
sol_df["component_id"] = sol_df["component_id"].str.strip('[]')
sol_df["sub_component_id"] = sol_df["sub_component_id"].str.strip('[]')

# Join compound IDs into one column, and remove duplicate IDs
sol_df['Compounds'] = sol_df[['component_id','sub_component_id']].apply(lambda x: ', '.join(x[x.notnull()]), axis = 1)
sol_df['Compounds'] = sol_df['Compounds'].apply(lambda x: ', '.join(sorted(set(x.split(', ')))))
sol_df = sol_df[["solution_id","Compounds"]]

# Split and explode on solution_id, regroup by Compound
sol_df = sol_df.assign(Compounds=sol_df['Compounds'].str.split(', ')).explode('Compounds')
cpd_sol = sol_df.groupby('Compounds')['solution_id'].agg(list).reset_index()

# Formatting
cpd_sol["solution_id"] = cpd_sol["solution_id"].astype(str)
cpd_sol["solution_id"] = cpd_sol["solution_id"].str.strip('[]')
cpd_sol = cpd_sol.rename(columns={"Compounds": "Compound ID"})
cpd_sol = cpd_sol.iloc[1:].reset_index(drop=True) # with parsing = '', e.g., solutions without subsolutions
filter = cpd_sol["Compound ID"].str.contains("None") # solutions recorded, but with no data filled
cpd_sol = cpd_sol[~filter]

cpd_sol.head()

,Compound ID,solution_id
0,1,"1, 3, 4, 7, 8, 21, 57, 64, 136, 145, 171, 173,..."
1,10,"5, 6, 12, 13, 14, 19, 48, 56, 58, 64, 67, 70, ..."
2,1000,"2252, 2253, 2269, 2821, 3560, 3628, 3632, 3634..."
3,1002,"2252, 2253, 2821, 4041, 4081, 4089, 4091, 4151..."
4,1005,2260


In [29]:
cpd_sol['Compound ID'] = cpd_sol['Compound ID'].astype("float64")
merged_df = pd.merge(left=compounds, right=cpd_sol, on="Compound ID", how="left")
merged_df = merged_df.rename(columns={"solution_id": "Used in Solutions:"})

merged_df.to_csv("~/Desktop/MediaDive_components.csv")
merged_df.head()

,Compound ID,Name,Group,Element,ChEBI,CAS Number,KEGG-ID,Mass [g/mol],Formula,Density,Used in Solutions:
0,1,Peptone,Peptone,Undefined,NaN,73049-73-7,NaN,NaN,NaN,NaN,"1, 3, 4, 7, 8, 21, 57, 64, 136, 145, 171, 173,..."
1,2,Meat extract,Meat extract,Undefined,NaN,NaN,NaN,NaN,NaN,NaN,"1, 3, 4, 12, 13, 14, 79, 80, 163, 171, 205, 49..."
2,3,Agar,Agar,Undefined,2509.0,9002-18-0,C08815,0.0000,(C12H18O9)n,NaN,"1, 3, 4, 5, 6, 7, 8, 9, 10, 11, 15, 16, 18, 19..."
3,4,Distilled water,Distilled water,Undefined,15377.0,7732-18-5,C00001,18.0153,H2O,1.00293,"1, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 16..."
4,5,Glucose,Glucose,NaN,17234.0,50-99-7,C00293,180.1600,C6H12O6,1.54400,"5, 7, 8, 11, 12, 13, 14, 16, 18, 21, 64, 68, 6..."
